In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1.inset_locator import (inset_axes, InsetPosition,
                                                  mark_inset)
from scipy.stats import binned_statistic_2d
import time as time
from scipy.stats import binom, skew
import pickle
from scipy.optimize import curve_fit
from matplotlib.cm import get_cmap
from scipy.fft import fft, ifft
import pandas as pd
from datetime import datetime
import sys
import ast
import os
from decimal import Decimal 
import scipy.linalg as scl
from numpy.linalg import det
from scipy.special import sici
from scipy.integrate import quad as integrate
import itertools 
from scipy.signal import periodogram as PS
from scipy.signal import welch

Helper functions for saving and representation

In [2]:
def phi_label_func(phi):
    if phi == np.pi:
        phi_label = r'$\pi$'
    elif phi == np.pi/2:
        phi_label = r'$\frac{\pi}{2}$'
    elif phi == np.pi/4:
        phi_label = r'$\frac{\pi}{4}$'
        
    elif phi == np.pi/3:
        phi_label = r'$\frac{\pi}{3}$'
        
    elif phi == 3*np.pi/4:a
        phi_label = r'$\frac{3\pi}{4}$'
    elif phi == 0:
        phi_label = '0'
    else:
        phi_label = str(np.round(phi,2))
    return phi_label

def create_spec_dict(T=1,t_seq = 3, lag = 3, N = 100000, omega = 0.01, D=1, NTLS = 10,
                    alpha = 0.75, W=0.01, V = 1, reps = 200, phi = np.pi/4):
    specs_dict = {'T' : T, 't_seq' : t_seq, 'lag' : lag, 'N' : N, 'reps' : reps, 'NTLS' : NTLS,
                 'D' : D, 'omega' : omega, 'alpha' : alpha, 'W' : W, 'V' : V, 'phi' : phi}
    return specs_dict

Defining correlation functions. For the centered correlation function we use the following definition:

For an array of outcomes $x$ of size ($m, M$) (for simplicity let's assume that it's a 1-D array, so we use $m=1$)

$$
C(k) =  \left\langle (x[:M-k] - \langle x[:M-k]\rangle) (x[k:]-\langle x[k:]\rangle)\right\rangle,
$$
where 
$$\langle x[:M-k]\rangle := \frac{1}{M-k}\sum_{i=0}^{M-k-1} x_i,$$
is an average over all outcomes of the sublist defined by the indices. Note, that the average in the first component, and the second one are different, and both are different than the total average, i.e. $r_1$. This allows us to get rid of statistical biases caused by a finite length of the arrays.
For 2-D arrays, we do the same, but the average is taken for each repetition. This is done in parallel through vectorization procedure. 
Next, $C(k)$ is aranged into a list [$C(k)$ for k in range(1,N)], where $N$ is the terminanting value of the list, that in princple can be as big as $M$. However, to avoid statistical uncertaintites, it is usually taken to be $N\le 1/\sqrt{M}$. Such composed list is what we call $\tilde{r}_2(k)$ in the paper. 

Analogously, we define three-time correlator $C(k,l)$ (i.e. $\tilde{r}_3(k,l)$), but instead of having two lists, we use three of them, and fix either $k$ or $l$ index to iterate over just one (for efficiency). 

In [3]:
def correlation(output, N = 0):
    output = np.array(output)
    if N == 0:
        N = output.shape[-1] 
    M = output.shape[-1] 
    m = output.shape[0]
    
    
    if len(output.shape) == 1:
        
        C_out = [np.mean((output[:M-k]- np.mean(output[:M-k])) * (output[k: ]- np.mean(output[k:]))) for k in range(N)]
    else:
        pi_1 = np.reshape(np.mean(output, axis = 1), (m,1))
        C_out = [np.mean((output[:,:M-k]- np.reshape(np.mean(output[:,:M-k], axis = 1), (m,1)) ) * 
                        (output[:,k: ]-  np.reshape(np.mean(output[:,k:], axis = 1), (m,1))), axis = 1) for k in range(N)]
    return C_out


def correlation_3_shift(output, lag, lag_type = 'k', N = 0):
    output = np.array(output)
    
    if N == 0:
        N = output.shape[-1] 
    M = output.shape[-1] 
    m = output.shape[0]

    tin = time.monotonic()
    if len(output.shape) == 1:
        if lag_type == 'k':
            C_out = [np.mean((output[:-k-lag]-np.mean(output[:-k-lag])) * 
                             (output[k:-lag ]-np.mean(output[k:-lag ])) * 
                             (output[k+lag: ]-np.mean(output[k+lag: ])))
                     for k in range(1,N)]
        elif lag_type == 'l':
            C_out = [np.mean((output[:-lag-k]) * (output[lag:-k]) * (output[lag+k: ])) 
                     for k in range(1,N)]   

    else:
        if lag_type == 'k':
            C_out = [np.mean((output[:,:-k-lag]-np.reshape(np.mean(output[:,:-k-lag], axis = 1), (m,1))) * 
                             (output[:,k:-lag ]-np.reshape(np.mean(output[:,k:-lag ], axis = 1), (m,1))) * 
                             (output[:,k+lag: ]-np.reshape(np.mean(output[:,k+lag: ], axis = 1), (m,1))), axis = 1) 
                     for k in range(1,N)] 
        elif lag_type == 'l':
            C_out = [np.mean((output[:,:-(k+lag)]-np.reshape(np.mean(output[:,:-(k+lag)], axis = 1), (m,1))) * 
                             (output[:,lag:-k]-np.reshape(np.mean(output[:,lag:-k], axis = 1), (m,1))) * 
                             (output[:,lag+k: ]-np.reshape(np.mean(output[:,lag+k:], axis = 1), (m,1))), axis = 1) 
                     for k in range(1,N-lag)]    
        C_out = np.array(C_out).T
    return C_out

## Functions for simulation of TLSs

In [4]:
def multiple_asymmetric(N, W12_list, W21_list, T, t_cycle,   dt, V_list, reps):
    '''
    
    Input:
    N       - number of repetitions of the experiment - Ramsey cycles
    W12     - noise/correlation parameter characterizing transition from state 1 --> 2 (in paper 1->0)
    W21     - as above, but the transition from 2 --> 1  (in paper 0->1)
    T       - time between Ramsey pulses (in paper t_R)
    t_cycle - total time for a single measurement (T + measurement and rest time) (in paper t_cyc)
    dt      - the minimal time increment, in which flipping between state 1 and 2 is happening 

    Return: an array of TLS fluctuation outcomes: "-V_n" and "+V_n"
    '''
    W12_array = np.array(W12_list)
    W21_array = np.array(W21_list)
    W = W12_array + W21_array
    
    n = len(W12_list)
    V_arr = np.reshape(np.array(V_list), (n,1))
    
    #random initial state either +1 or -1
    dn = np.zeros((reps, n, int(t_cycle * N/dt)))  #this might need to be adjusted
    dn[:,:,0] = (np.random.randint(2, size = (reps, n))*2 -1)
    
    #switching probabilities for the smallest time increments
    p12 = W12_array * dt
    p21 = W21_array * dt
    
    flip = 0
    for idx in range(int(t_cycle * N/dt) -1):
#         print('idx ', idx)
        rnd = np.random.rand(reps,n)
        I12 = (-2*(rnd < p12) +1)
        I21 = (-2*(rnd < p21) +1)
#         print(f'I12 : {I12}, I21 : {I21}')
        d21 = dn[:,:,idx] == +1
        d12 = dn[:,:,idx] == -1
        dn[:,:,idx+1] = (-(d12 * I12) + d21 * I21)

    return dn*V_arr


def integrate_TLS(outcome_arr, N, T, t_cycle, dt):
    '''
    Function that transforms TLS fluctuations into a random phase theta, through sumation
    
    Input:
    outcome_arr - results from TLS fluctuators, e.g. output of multiple_asymmetric_list function
    N - number of Ramsey cycle
    T       - time between Ramsey pulses
    t_cycle - total time for a single measurement (T + measurement and rest time)
    dt      - the minimal time increment, in which flipping between state 1 and 2 is happening
    
    Return: an array of phases, that allow us to define probability of an outcome
    '''
    
    outcome_arr = np.array(outcome_arr)
#     print('shape : ', outcome_arr.shape)
    t_steps = int(t_cycle/dt)
    theta = [dt *  np.sum(outcome_arr[:, :,k * t_steps: k*t_steps + int(T/dt)], axis = (1,2))
             for k in range(N) ]
    return np.array(theta).T


def outcomes_measurements(theta_array, phi = 0):
    '''
    function that computes outcomes of the measurements based on the random thetas
    '''
    dim_0 = theta_array.shape[0]
    dim_1 = theta_array.shape[1]
    rnd = np.random.rand(dim_0, dim_1)

    prob = 1/2 * (1 + np.cos(phi + theta_array))
    return (prob >= rnd) *1


def noise_stat_freq(N_total, W12_list, W21_list,T, t_cycle, dt, V_list, reps, phi = 0, avg = True):
    '''
    function that aggregates statistics of the outcomes
    It allows us to compute multiple repetetion of the procedure (parameter reps)
    
    '''
    dn = multiple_asymmetric(N = N_total, W12_list = W12_list, W21_list=W21_list, 
                               T = T, t_cycle = t_cycle, dt = dt, V_list = V_list, reps = reps)
    theta = integrate_TLS(dn, N=N_total ,T= T, t_cycle = t_cycle,  dt=dt)
    W, dW = delta_W(W12_list, W21_list)
    V_arr = np.array(V_list)
    theta_shift = T* np.sum(V_arr*dW/W)
    outcomes_01 = outcomes_measurements(theta, phi = phi - theta_shift)
    if avg:
        dn = np.sum(dn, axis = 1)
    return theta, outcomes_01, dn

Helper functions to calculate $r_1$ and $r_2(k)$

In [5]:
def delta_W(W12_list, W21_list):
    '''
    Calculates W = W_12 + W_21, and  ΔW = W_12 - W_21
    Note that 2 is 0 papers notation and 1 is 1 (the change is due to old convention) 
    '''
    W12_arr = np.array(W12_list)
    W21_arr = np.array(W21_list)
    dW_arr = W12_arr - W21_arr
    W_arr = W12_arr + W21_arr
    return W_arr, dW_arr

def delta_nu(W12_list, W21_list, V_list):
    W_arr, dW_arr = delta_W(W12_list, W21_list)
    V_arr = np.array(V_list)
    nu = 1/2 * np.sqrt(W_arr**2 + 4j*V_arr*(dW_arr +1j*V_arr))
    return nu

def pi1_TLS(W12_list, W21_list, T, V_list, phi = 0):
    '''
    This is what in the paper is called $r_1$
    '''
    V_arr = np.array(V_list)
    W_arr, dW_arr = delta_W(W12_list, W21_list)
    K_arr = dW_arr/W_arr
    nu = delta_nu(W12_list, W21_list, V_list)
    re = np.exp(-W_arr*T/2)/nu * ((W_arr/2 + 1j*V_arr*K_arr)*np.sinh(nu*T)+nu*np.cosh(nu*T))
    return 1/2 + 1/2 * np.real(np.exp(1j*phi)*np.prod(re))
    


In [6]:
def C_plus0(W12_list, W21_list, T, t_cycle, V_list, k=1):
    W_arr, dW_arr = delta_W(W12_list, W21_list)
    nu = delta_nu(W12_list, W21_list, V_list)
    V_arr = np.array(V_list)
    return 1j*V_arr/(nu)*np.exp(-W_arr*(k*t_cycle - T/2))*np.sinh(nu*T)

def C_plus0T(W12_list, W21_list, T, V_list):
    W_arr, dW_arr = delta_W(W12_list, W21_list)
    nu = delta_nu(W12_list, W21_list, V_list)   
    V_arr = np.array(V_list)
    K_arr = dW_arr/W_arr 
    Q = (W_arr/2 + 1j*V_arr*K_arr)*np.sinh(nu*T)+nu*np.cosh(nu*T)
    return 1/(2*nu)*np.exp(-W_arr*T/2)*Q
    
def J_plus(W12_list, W21_list, T, V_list):
    W_arr, dW_arr = delta_W(W12_list, W21_list)
    nu = delta_nu(W12_list, W21_list, V_list) 
    V_arr = np.array(V_list)
    K_arr = dW_arr/W_arr 
    W12_arr = np.array(W12_list)
    W21_arr = np.array(W21_list)
    return 2j*V_arr/nu * W12_arr*W21_arr/(W_arr**2) * np.exp(-W_arr*T/2)*np.sinh(nu*T)

def K_ms(W12_list, W21_list, T, t_cycle, V_list, idx = 0, k=1):
    J_p = J_plus(W12_list, W21_list, T, V_list)[:idx+1]
    C_p = 2*C_plus0T(W12_list, W21_list, T, V_list)[idx+1:]
    W_arr, dW_arr = delta_W(W12_list, W21_list)
    decay = np.exp(-np.sum(W_arr)*(k*t_cycle - T))
    return (np.prod(J_p*C_p) + np.conj(np.prod(J_p*C_p)))*decay
    
    
def pi1k(W12_list, W21_list, T, t_cycle, V_list, k=1, phi = 0, disp = False):
    
    pi11 = 0
    n = len(W12_list)
    C_plus = 2*C_plus0(W12_list, W21_list, T, t_cycle, V_list, k=k)
    C_pT = 2*C_plus0T(W12_list, W21_list, T, V_list)
    
    J_p = J_plus(W12_list, W21_list, T, V_list)    
    W_arr, dW_arr = delta_W(W12_list, W21_list)
    if disp:
        print(f'W : {W_arr}, dW : {dW_arr}')
    idx_list = np.arange(n)[::-1]
    idx_set = set(idx_list)
    for s in range(n):
        K_idx_list = itertools.combinations(idx_list, s+1)
        for K_idx in K_idx_list:
            n_list = list(idx_set.difference(set(K_idx)))
            J_list = [J_p[k] for k in K_idx]
            W_list = [W_arr[k] for k in K_idx]
            if len(n_list) == 0:
                C_p_list = [1]
            else:
                C_p_list = [C_pT[k] for k in n_list]
            C_p_m = [C_plus[k] for k in K_idx]
            if disp:
                print(f'J : {J_list}, C : {C_p_list}, Cpm : {C_p_m}')
                print(f's : {s}, ms : {K_idx}, n : {n_list}')
            K = 1/4*np.prod(J_list)*np.prod(C_p_list)*np.exp(1j*phi)
            K = K + np.conj(K)
            pi11 += 1/2*K*np.prod(C_p_m)*np.prod(C_p_list)
        
    return np.real(pi11*np.exp(1j*phi))

def pi11_list(W12_list, W21_list, T, t_cycle, V_list,  phi = 0, k_max = 100):
    '''
    This is a function that calculates $\tilde{r}_2(k)$ - centered correlator according to Eq. (19)
    in the paper. The subroutines of that functions are different than the one presented in the paper,
    however they give the same results (old notation and derivation convention)
    '''
    
    
    Pi11_list = [pi1k(W12_list, W21_list, T, t_cycle, V_list, k=k, phi = phi) for k in range(1,k_max)]
    return Pi11_list

Main sampling function that also saves the results for a qubit dispersively coupled to a set of TLSs

In [8]:
def TLS_sampling_PS(N_total, W12_list, W21_list, T, t_cycle, dt, V_list,  phi=0, 
                          lag_list = [0], reps=1, save = True, k_max = 0, 
                    avg = True, W_type = '', save_all = True, PS = True,
                   batch_size = 0):
    '''
    Function that simulates, calculates, and saves  TLS evolution
    average is required to be true if we want to calculate PS
    save_all is also required if additionally we're batching
    
    Input: 
    N_total   -- number of Ramsey cycles in one repetition
    W12_list  -- switching rates between 1 and 2 (1 and 0 in the paper's notation)
    W21_list  -- switching rates between 2 and 1 (0 and 1 in the paper's notation)
    T         -- interpulse time (t_R in the paper)
    t_cycle   -- interpulse + reset time (t_cyc)
    dt        -- discrete time step, at whcih TLSs switching can happen
    V_list    -- list of coupling strengths
    phi       -- phase shift (detuning)
    lag_list  -- a list of "lags"/"delays" in r_3(k,l) (i.e. fixing value of k or l at the value of a lag)
                 if you want to calculate r_3(k, k+3), you select lag_list = [3]. If a list is longer,
                 it allows to calcluate more setups (here we fixed lags only to k, easy to change though)
    reps      -- how many times an entire sequence of measurements is repeated
    save      -- True if we want to save
    k_max     -- how many correlation elements we want to compute
    avg       -- averages out all frequency fluctuations over TLSs ('vertical' integration over TLS average, contrary
                 to a 'horizontal' integration (in time) to obtain theta)
    W_type    -- a string for saving, I use here 'sym'/'asym' to indicate if the setup is composed of symmetric
                 asymmetric TLSs, but it can be any string that is helpful.
    save_all  -- saves all data: all TLSs outcomes, theta, correlations etc. Note that this generates large files
    PS        -- if we want to keep track of the empirical power spectrum (FFT)
    batch_size -- splits the sampling into the number of batches and than snitches back data together,
                 allows to avoid keeping large arrays in the memory.
                 
    '''
    t0 = time.monotonic()
    t_stamp = datetime.now()
    
    W_arr, dW_arr = delta_W(W12_list, W21_list)
    V0 = V_list[0]
    theta_list = []
    outcomes_list = []
    dn_list = []
    NTLS = len(W12_list)

    tin = time.monotonic()
    
    
    if batch_size == 0:
        
        theta, outcomes_01, dn = noise_stat_freq(N_total, W12_list, W21_list,T, t_cycle, 
                                        dt,V_list = V_list, phi = phi, avg = avg, reps = reps )
    else:
        if save_all:
            theta = np.zeros((reps,N_total))
            dn = np.zeros((reps, NTLS, N_total))
        outcomes_01 = np.zeros((reps, N_total))
        for k in range(batch_size):
            reps_new = reps//batch_size
            if save_all:
                theta_temp, outcomes_temp, dn_temp = noise_stat_freq(N_total, W12_list, W21_list,T, t_cycle, 
                                                dt,V_list = V_list, phi = phi, avg = avg, reps = reps_new )    
                theta[k*reps_new:(k+1)*reps_new,:] = theta_temp
                if avg:
                    dn[k*reps_new:(k+1)*reps_new,:] = dn_temp
                else:
                    dn[k*reps_new:(k+1)*reps_new,:,:] = dn_temp
            else:
                _, outcomes_temp, _ = noise_stat_freq(N_total, W12_list, W21_list,T, t_cycle, 
                                dt,V_list = V_list, phi = phi, avg = avg, reps = reps_new )
            outcomes_01[k*reps_new:(k+1)*reps_new,:] = outcomes_temp
            print(f'time for batch {k}: {time.monotonic() - tin}')
                
    
    if PS:
        m = dn.shape[-1]
        Sf = np.abs(fft(dn)*dt)**2 / (m*dt)
        Sf = Sf/reps
        freq = np.arange(m)/(dt*m)
    else:
        Sf = 0
        freq = 0    


    tin = time.monotonic()
    if k_max == 0:
        k_max = int(np.sqrt(N_total))+10
    pi1_sampled = np.mean(outcomes_01)
    pi1_theo = pi1_TLS(W12_list, W21_list, T, V_list, phi = phi)

    n = len(W12_list)
    print(f'pi(1) sampled: {pi1_sampled}, pi(1) theo : {pi1_theo}')
    W_min = W_arr[0]
    
    if W_min == 0.01:
        W_idx = '0'
    elif W_min == 0.001:
        W_idx = '1'
    elif W_min == 0.0001:
        W_idx = '2'
    else :
        W_idx = '00'
        
    if phi == 0:
        phi_idx = '0'
    elif phi == np.pi/2:
        phi_idx = '1'
    elif phi == np.pi/4:
        phi_idx = '2'
    else: 
        phi_idx = '00'
    


    if save:
        parent = f'./data_TLS'
        if not os.path.exists(parent):
            os.mkdir(parent)
            
        path = parent +  f'/data_T={T}_tseq={t_cycle}_W={W_idx}_phi={phi_idx}_N={N_total}_NTLS={NTLS}_V={V0}_alpha={alpha}{W_type}'

        if not os.path.exists(path):
            os.mkdir(path)
            
        specs_dict = {"N_total" : N_total, 
                      "W12_list" : W12_list,
                     "W21_list" : W21_list,
                      "V_list" : V_list,
                     "T" :T, "t_cycle" : t_cycle, 'dt' : dt, 
                      'phi' : phi, 'k_max' : k_max,
                     'reps' : reps,
                     "N-TLS" : len(W12_list), 
                      'pi1_samp' : pi1_sampled, 'pi1_theo' : pi1_theo} 

        file = open(path + "/specs.txt","w")
        for key, value in specs_dict.items():     
            file.write('%s:%s\n' % (key, value))
        file.close()

    Pi11 = np.array(correlation(outcomes_01, N = k_max)).T
    if NTLS <= 20:
        Pi11_theo = np.array(pi11_list(W12_list, W21_list, T, t_cycle, V_list,  phi = phi, k_max = k_max))
    else:
        Pi11_theo = [0]
    print('time ellapsed C_out: ', time.monotonic() - tin)
    
    if lag_list[0] != 0 :
        C3_list = []
        Pi111_list = []
        tin3 = time.monotonic()
        for lag in lag_list:
            C3 = correlation_3_shift(outcomes_01, lag, lag_type = 'k', N = k_max + 15)
            C3_list.append(np.mean(C3, axis = 0))

            

    r2 = np.mean(Pi11, axis = 0)
    print(f'r2 shape : {r2.shape}')
    if phi != np.pi/2:
        exp_f11 = (2*pi1_sampled-1)/np.cos(phi)
        if exp_f11 < 0.5:
            f11 = 0
        else:
            f11 = -2*np.log(exp_f11)
    elif phi == np.pi/2:
        print("Can't determine f_0!")
        exp_f11 = 1
        f11 = 0

    if phi == np.pi/4:
        exp_f1k = 8*r2[1:]/(exp_f11**2) + 1
        f1k = np.log(exp_f1k)
    elif phi == 0:
        cosh_f1k = 4*r2[1:]/(exp_f11**2) + 1
        f1k = np.arccosh(cosh_f1k)
    elif phi == np.pi/2:
        sinh_f1k = 4*r2[1:]/(exp_f11**2) 
        f1k = np.arcsinh(sinh_f1k)

    F_row = np.zeros(f1k.shape[-1]+1)
    F_row[0] = f11
    F_row[1:] = f1k
    r3_theo = pi1kl_shift_list(F_row, 3, phi=phi, lag_type = 'k')
            
    specs_dict = {"N_total" : N_total, 
              "W12_list" : W12_list,
             "W21_list" : W21_list,
              "V_list" : V_list,
             "T" :T, "t_cycle" : t_cycle, 'dt' : dt, 
              'phi' : phi, 'k_max' : k_max,
             'reps' : reps,
             "N-TLS" : len(W12_list), 
              'pi1_samp' : pi1_sampled, 'pi1_theo' : pi1_theo} 
    
    if save_all:
        print('save all')
        output_dict = {'dn' :dn, 'out': np.array(outcomes_01), 'pi11_samp': Pi11, 
              'pi11_theo': Pi11_theo, 'pi111' :C3_list, 'theta':theta_list, 
                   'pi1':[pi1_sampled, pi1_theo], 'specs' : specs_dict, 
                       'freq' : freq, 'Sf' : Sf, 'pi111_theo' : r3_theo, 'Frow': F_row }
    else:
        output_dict = {'dn' :0,'out': np.array(outcomes_01), 'pi11_samp': Pi11, 
              'pi11_theo': Pi11_theo, 'pi111' :C3_list, 'theta':0, 
                   'pi1':[pi1_sampled, pi1_theo], 'specs' : specs_dict,
                       'freq' : freq, 'Sf' : Sf, 'pi111_theo' : r3_theo, 'Frow': F_row}        
    if save: 

        output_dict_to_dump = f'/output_dict.p'
        with open(path + output_dict_to_dump, 'wb') as f:
            pickle.dump(output_dict, f)  

    return output_dict

Similar routine as above, but tailored to collect only samples for short sequences, to study non-ergodic behavior

In [9]:
def TLS_sampling_short(N_total, W12_list, W21_list, T, t_cycle, dt, 
                       V_list,  phi=0, reps=1, save = False, batch_size = 0, stamp = ''):
    '''
    Function that simulates, calculates, saves and plots TLS evolution
    '''
    t0 = time.monotonic()
    t_stamp = datetime.now()
    
    W_arr, dW_arr = delta_W(W12_list, W21_list)
#     V0 = V_list[0]

    if batch_size == 0:
        
        theta, outcomes_01, dn = noise_stat_freq(N_total, W12_list, W21_list,T, t_cycle, 
                                        dt,V_list = V_list, phi = phi, avg = True, reps = reps )
    else:
        if theta_save:
            theta = np.zeros((reps,N_total))
#             dn = np.zeros((reps, NTLS, N_total))
        outcomes_01 = np.zeros((reps, N_total))
        for k in range(batch_size):
            reps_new = reps//batch_size
            if theta_save:
                theta_temp, outcomes_temp, dn_temp = noise_stat_freq(N_total, W12_list, W21_list,T, t_cycle, 
                                                dt,V_list = V_list, phi = phi, avg = True, reps = reps_new )    
                theta[k*reps_new:(k+1)*reps_new,:] = theta_temp

            else:
                _, outcomes_temp, _ = noise_stat_freq(N_total, W12_list, W21_list,T, t_cycle, 
                                dt,V_list = V_list, phi = phi, avg = True, reps = reps_new )
            outcomes_01[k*reps_new:(k+1)*reps_new,:] = outcomes_temp
        

    print(f'time: {time.monotonic() - t0}')
    
    if save:
        parent = f'./data_TLS_short'
        if not os.path.exists(parent):
            os.mkdir(parent)
    
        W_min = W_arr[0]

        if W_min == 0.01:
            W_idx = '0'
        elif W_min == 0.001:
            W_idx = '1'
        elif W_min == 0.0001:
            W_idx = '2'
        else :
            W_idx = '00'

        if phi == 0:
            phi_idx = '0'
        elif phi == np.pi/2:
            phi_idx = '1'
        elif phi == np.pi/4:
            phi_idx = '2'
        else: 
            phi_idx = '00'
    
        path = parent +  f'/data_T={T}_tseq={t_cycle}_W={W_idx}_phi={phi_idx}_N={N_total}_NTLS={NTLS}_V={V0}_alpha={alpha}_{stamp}'

        if not os.path.exists(path):
            os.mkdir(path)
            
        

        output_dict_to_dump = f'/outcomes.p'
        with open(path + output_dict_to_dump, 'wb') as f:
            pickle.dump(outcomes_01, f)  
            
        specs_dict = {"N_total" : N_total, 
              "W12_list" : W12_list,
             "W21_list" : W21_list,
              "V_list" : V_list,
             "T" :T, "t_cycle" : t_cycle, 'dt' : dt, 
              'phi' : phi, 'reps' : reps,
             "N-TLS" : len(W12_list)} 

        file = open(path + "/specs.txt","w")
        for key, value in specs_dict.items():     
            file.write('%s:%s\n' % (key, value))
        file.close()        
        
    return np.array(outcomes_01)

## Functions to simulate Gaussian noise

Helper functions

In [10]:
def expcosh(a,b):
    return (np.exp(-(a+b)) + np.exp(-a + b))/2

def pi1(f11, phi = 0):
    return 1/2 * (1 + np.cos(phi) * np.exp(- f11 / 2))

def pi11(f11, f1k, phi = 0):
    return 1/8 * np.exp(-f11) * (np.exp(f1k) - 1 - np.cos(2*phi) * (1-np.exp(-f1k)))

def pi111(f11, f12, f13 ):
    return pi11(f11, f12) + 1/2 * pi11(f11,f13) -3/4 * pi1(f11) + 1/8 * (1 + 1/2 * np.exp(-3/2 * f11) * (expcosh(f13, 2*f12) + np.exp(f13)))


def full_pi11(F_row, phi = 0):
    f11 = F_row[0]
    N = len(F_row)
    Pi11 = [pi11(f11, F_row[k], phi = phi) for k in range(1,N)]
    return Pi11


def pi1kl_f(f1k, f1l, fkl, f11, phi = 0):

    pi_1 = pi1(f11, phi = phi)
    pi1k = pi11(f11, f1k, phi = phi)
    pi1l = pi11(f11, f1l, phi = phi)
    pikl = pi11(f11, fkl, phi = phi)
    Q1 = (np.exp(-f1k -f1l -fkl) - 1)* np.cos(3*phi)
    Q2 = np.cos(phi) * (np.exp(-f1k+f1l+fkl) + np.exp(-f1l +f1k +fkl) + np.exp(f1k+f1l-fkl) -3)
    return 1/32 * np.exp(-3/2 * f11) * (Q1+Q2)

def pi1kl_f_list(F_row,  lag, phi = 0, lag_type = 'k'):
    N_samp = len(F_row)
    f11 = F_row[0]
    if lag_type == 'k':
        C_out = [pi1kl_f(F_row[lag], F_row[lag+k], F_row[k], f11, phi = phi) for k in range(1,N_samp-lag)]
    elif lag_type == 'l':
        C_out = [pi1kl_f(F_row[k], F_row[lag+k], F_row[lag], f11, phi = phi) for k in range(1,N_samp-lag)]
    return C_out

def pi1kl_shift(f1k, f1l, fkl, f11, phi = 0):
    Q1 = np.cos(phi) * (np.exp(-f1k+f1l+fkl) + np.exp(-f1l +f1k +fkl) + np.exp(f1k+f1l-fkl) +
                        6-2*np.exp(f1k)-2*np.exp(f1l)-2*np.exp(fkl)-
                        np.exp(-f1k)-np.exp(-f1l)-np.exp(-fkl))
    Q2 = np.cos(3*phi)*(np.exp(-f1k -f1l -fkl) +2-np.exp(-f1k)-np.exp(-f1l)-np.exp(-fkl) )
    return 1/32 * np.exp(-3/2 * f11) * (Q1+Q2)
def pi1kl_shift_list(F_row, lag, phi=0, lag_type = 'k'):
    N_samp = len(F_row)
    f11 = F_row[0]
    if lag_type == 'k':
        C_out = [pi1kl_shift(F_row[lag], F_row[lag+k], F_row[k], f11, phi = phi) for k in range(1,N_samp-lag)]
    elif lag_type == 'l':
        C_out = [pi1kl_shift(F_row[k], F_row[lag+k], F_row[lag], f11, phi = phi) for k in range(1,N_samp-lag)]
    return C_out

Correlation matrix $f_k=\langle \theta_0\theta_k\rangle$ for exponentially correlated noise. Here we use slightly different representation that is currently in the paper. However, easy to map one to another one.

In [11]:
def fnm(T, D, tn, tm):
    return D * (np.cosh(T)-1) * np.exp(-abs(tn - tm))

def f_one_one(T, D):    
    return D * (T - (1-np.exp(-T)))


def F_matrix(N, T, D, t_cycle):
    f11 = f_one_one(T,D)
    F = np.array([[fnm(T, D,k * t_cycle, l * t_cycle)  if k != l else f11 for k in range(N)] for l in range(N)])
    return F

def Fnm_row(N, T, D, t_cycle):
    f11 = f_one_one(T,D)
    F = np.array([fnm(T,D,0,k*t_cycle) if k != 0 else f11 for k in range(N)])
    return F

Correlation function for $1/f$ noise 

In [12]:
def fk_approx(N, D, T, t_cycle, omega):
    gamma = np.euler_gamma
    k_start = 150
    k_list = np.arange(1,N+1)
    fk = D*T**2/(np.pi)*(-gamma - np.log(k_list*omega*t_cycle))
    return fk

def f0_new(T, D, omega):
    b = omega * T
    shi, chi = shichi(b)
    return T**2 * D/np.pi * (-1/b**2 + 2/b- (b-1)*np.exp(-b)/b**2 - chi+shi )

def f_one_one_arctan(T, D, omega_min):
    b = omega_min * T   
    return D * 1/2 * (-(1/b**2) + 2/b - ((-1 + b) * np.exp(-b))/b**2 
                     - shichi(b)[1] +    shichi(b)[0])
    

def fnm_arctan(T, D, omega_min, t_cycle, n):
    a = n*t_cycle/T
    b = omega_min * T

    return D*T**2/(2*np.pi)* (2 * a**2 * expi(-a * b) + 1/b**2*np.exp(-a * b) * 
                       (-2 * (-1 + a * b) * (-1 + np.cosh(b))  
          - b**2 * np.exp(a * b) * ((1 + a)**2 * expi(-((1 + a) * b)) + (-1 + 
            a)**2 * expi(b - a * b)) + 2 * b * np.sinh(b)))

def F_matrix_arctan(N, T, D, t_cycle, omega_min):
    f11 = f0_new(T, D, omega_min)
    F = np.array([[fnm_arctan(T, D, omega_min,  t_cycle, abs(k-l))  if k != l else f11 for k in range(N)] for l in range(N)])
    return F

def Fnm_row_arctan(N,  T , D, t_cycle, omega_min):
    f11 = f0_new(T, D, omega_min)
    F = np.array([fnm_arctan( T, D, omega_min,  t_cycle, k) if k != 0 else f11 for k in range(N)])
    return F

Sampling function of random phases. First for exponentially correlated, then for 1/f noise

In [13]:
def sampling_theta(N_samples, T, D,  t_cycle,  n_corr, reps):
    

    F = F_matrix(n_corr+1, T, D,  t_cycle)

    for N in range(2,n_corr):
        k = N-1
        F_old = F[:k,:k]
        Psi_old = np.linalg.inv(F_old)
        M = Psi_old[-1]
        if np.any(abs(M)  < 1e-4):
            n_corr = N
            break
    if n_corr <=2 :
        n_corr = 2
    print(f'n_corr : {n_corr}')
    tin = time.monotonic()
    F = F_matrix(n_corr+1, T, D,  t_cycle)
    Psi = np.linalg.inv(F)
    print(f'F shape = {F.shape}')
    sigma = np.sqrt(1/Psi[0,0])

    
    theta_1 = np.random.normal(loc = 0, scale = sigma, size = (1, reps))
    theta_arr = np.zeros(( reps, N_samples))
    theta_arr[:,0] = theta_1
    
    sigma_list = [sigma]
    mu_list = []
    for k in range(1,N_samples):
        
        if k <= n_corr-1:
            Psi_temp = np.linalg.inv(F[:k+1,:k+1])
            sigma = np.sqrt(1/Psi_temp[k,k])
            mu = -np.sum(Psi_temp[k,:k]*theta_arr[:,:k], axis = 1)/Psi_temp[k,k]
            psi_theta = Psi_temp[k,:k]*theta_arr[:,:k]
            theta_tmp = np.random.normal(loc = mu, scale = sigma, size = (1,reps))
            theta_arr[:,k] = theta_tmp
            mu_list.append(mu)

        else:
            mu = -np.sum(Psi[n_corr,-n_corr-1:n_corr]*theta_arr[:,k-n_corr:k], axis = 1)/Psi[n_corr,n_corr]
            sigma = np.sqrt(1/Psi[n_corr,n_corr])
            theta_tmp = np.random.normal(loc = mu, scale = sigma, size = (1,reps))
            theta_arr[:,k] = theta_tmp
            mu_list.append(mu)
    return theta_arr, F

In [17]:
def sampling_theta_flicker(N_samples, T, D,  t_cycle, omega,  n_corr, reps, 
                           noise = 'int', progress = False,Gamma = 0.0005):
    '''
    noise parameter, Gamma are obsolete
    '''
    
    rt = t_cycle/T

    F = F_matrix_arctan(n_corr+1, T, D, t_cycle, omega)



    print(f'n_corr : {n_corr}')
    tin = time.monotonic()

    Psi = np.linalg.inv(F)
    print(f'F shape = {F.shape}')
    sigma = np.sqrt(1/Psi[0,0])

    theta_1 = np.random.normal(loc = 0, scale = sigma, size = (1, reps))
    theta_arr = np.zeros(( reps, N_samples))
    theta_arr[:,0] = theta_1
    
    sigma_list = [sigma]
    mu_list = []
    tin = time.monotonic()
    for k in range(1,N_samples):
        
        if k <= n_corr-1:
            Psi_temp = np.linalg.inv(F[:k+1,:k+1])
            sigma = np.sqrt(1/Psi_temp[k,k])
            mu = -np.sum(Psi_temp[k,:k]*theta_arr[:,:k], axis = 1)/Psi_temp[k,k]
            psi_theta = Psi_temp[k,:k]*theta_arr[:,:k]
            theta_tmp = np.random.normal(loc = mu, scale = sigma, size = (1,reps))
            theta_arr[:,k] = theta_tmp
            mu_list.append(mu)

        else:
            mu = -np.sum(Psi[n_corr,-n_corr-1:n_corr]*theta_arr[:,k-n_corr:k], axis = 1)/Psi[n_corr,n_corr]
            sigma = np.sqrt(1/Psi[n_corr,n_corr])
            theta_tmp = np.random.normal(loc = mu, scale = sigma, size = (1,reps))
            theta_arr[:,k] = theta_tmp
            mu_list.append(mu)
        if progress:
            if k % 2500 == 0:
                print(f'k = {k}, t : {time.monotonic() - tin}')
    return theta_arr, F

In [18]:
def full_sampling(N_samples, T, D, t_cycle, lag = 5, 
                  save = False, reps = 100, k_max = 0,  
                  n_corr = 20, N = 0, save_all = False, phi_list = [0, np.pi/4, np.pi/2]):
    
    specs_dict = {"T" : T, "N_samp" : N_samples, "D" : D, "t_cycle" : t_cycle, 'lag' : lag,
                 'reps' : reps, 'k_max' : k_max, 'n_corr' : n_corr, 'N' : N}
    
    t0 = time.monotonic()
    theta_dict = {}
    outcome_dict = {}
    Pi1_dict = {}
    Pi11_dict = {}
    Pi111_dict = {}
    Pi111_shift_dict = {}
    
    Pi1_theo_dict = {}
    Pi11_theo_dict = {}
    Pi111_theo_dict = {}
    Pi111_shift_theo_dict = {}

    F_predicted_dict = {}


    
    theta_arr, F = sampling_theta(N_samples = N_samples, T = T, D = D,
                                                t_cycle = t_cycle, n_corr =  n_corr,reps = reps)
    
    if k_max == 0:
        k_max = int(np.sqrt(N_samples))
    
    for idx, phi in enumerate(phi_list):
        
        rnd = np.random.rand(reps, N_samples)
        
        out = (1/2 * (1 + np.cos(phi + theta_arr)) >= rnd) * 1
        
        outcome_dict[idx] = out
        
        tin = time.monotonic()
        f11 = F[0,0]
        F_row = Fnm_row(N+10,T, D, t_cycle)

        
        Pi1_full = np.mean(out, axis = 1).reshape(-1,1)  
        Pi1 = np.mean(out)
        Pi1_theo = pi1(f11, phi)
        Pi1_dict[idx] = [Pi1, Pi1_full]
        Pi1_theo_dict[idx] = Pi1_theo
        
        Pi11 = (np.array(correlation(out, N)).T)[:,1:]
        Pi11_theo = full_pi_11(T,  D,  t_cycle = t_cycle, phi = phi,  k_max = N_samples)[1:]
        Pi11_dict[idx] = Pi11
        Pi11_theo_dict[idx] = Pi11_theo

        
        Pi111_dict[idx] = [0]
        Pi111_theo_dict[idx] =[0]
        
        Pi111_shift = correlation_3_shift(out, lag, lag_type = 'k', N = N+10)
        Pi111_shift_theo = pi1kl_shift_list(F_row, lag, phi, lag_type = 'k')
        
        Pi111_shift_dict[idx] = Pi111_shift
        Pi111_shift_theo_dict[idx] = Pi111_shift_theo
        

            
        print(f'idx : {idx} time : {time.monotonic() - t0}')
    if save_all:
        output = (outcome_dict, 0, Pi1_dict, Pi1_theo_dict, 
                  Pi11_dict, Pi11_theo_dict, Pi111_dict, Pi111_theo_dict, 
                  Pi111_shift_dict, Pi111_shift_theo_dict,
                 F_predicted_dict, F, specs_dict)
    else:
        output = (0, 0, Pi1_dict, Pi1_theo_dict, 
          Pi11_dict, Pi11_theo_dict, Pi111_dict, Pi111_theo_dict, 
          Pi111_shift_dict, Pi111_shift_theo_dict,
         0, 0, specs_dict)
    return output
        
    
def full_sampling_flicker(N_samples, T, D, t_cycle, omega, lag = 5, 
              save = False, reps = 100, k_max = 0,  n_corr = 20,
                          N = 0, noise = 'int', save_all = False, 
                          phi_list = [0, np.pi/4, np.pi/2], Gamma = 0.005):


    specs_dict = {"T" : T, "N_samp" : N_samples, "D" : D, "t_cycle" : t_cycle, 'lag' : lag,
                 'reps' : reps, 'k_max' : k_max, 'n_corr' : n_corr, 'N' : N, 'omega' : omega,
                 'noise' : noise}

    t0 = time.monotonic()
    theta_dict = {}
    outcome_dict = {}
    Pi1_dict = {}
    Pi11_dict = {}
    Pi111_dict = {}
    Pi111_shift_dict = {}

    Pi1_theo_dict = {}
    Pi11_theo_dict = {}
    Pi111_theo_dict = {}
    Pi111_shift_theo_dict = {}
    
    F_predicted_dict = {}

    

    theta_arr, F = sampling_theta_flicker(N_samples = N_samples, T = T, D = D,
                                                t_cycle = t_cycle, omega = omega,
                                          n_corr =  n_corr,reps = reps, 
                                          noise = noise, Gamma = Gamma)
    if k_max == 0:
        k_max = int(np.sqrt(N_samples))
    print(f'time for theta : {time.monotonic() - t0}, k_max : {k_max}')
    rt = t_cycle/T
    
    F_row = F[0]

    for idx, phi in enumerate(phi_list):

        rnd = np.random.rand(reps, N_samples)

        out = (1/2 * (1 + np.cos(phi + theta_arr)) >= rnd) * 1

        outcome_dict[idx] = out

        tin = time.monotonic()
        f11 = F[0,0]
        if noise == 'color':
            F_row = Fnm_row_color(N+10, T, D, t_cycle, omega, Gamma)
        else:
            F_row = Fnm_row_arctan(N+10,T, D, t_cycle, omega )


        Pi1_full = np.mean(out, axis = 1).reshape(-1,1)  
        Pi1 = np.mean(out)
        Pi1_theo = pi1(f11, phi)
        Pi1_dict[idx] = [Pi1, Pi1_full]
        Pi1_theo_dict[idx] = Pi1_theo
        
        print(f'time for Pi1 : {time.monotonic() - tin}')
        
        Pi11 = (np.array(correlation(out, N)).T)[:,1:]
        print(f'time for Pi11 samp : {time.monotonic() - tin}')
        Pi11_theo = full_pi11(F_row, phi = phi)
        Pi11_dict[idx] = Pi11
        Pi11_theo_dict[idx] = Pi11_theo
        print(f'time for Pi11 total: {time.monotonic() - tin}')

        Pi111_shift = correlation_3_shift(out, lag, lag_type = 'k', N = N+10)
        Pi111_shift_theo = pi1kl_shift_list(F_row, lag, phi, lag_type = 'k')
        
        Pi111_shift_dict[idx] = Pi111_shift
        Pi111_shift_theo_dict[idx] = Pi111_shift_theo

    if save_all:
        output = (outcome_dict, 0, Pi1_dict, Pi1_theo_dict, 
                  Pi11_dict, Pi11_theo_dict, 0, 0, 
                  Pi111_shift_dict,Pi111_shift_theo_dict,
                 0, 0, specs_dict)
    else:
        output = (0, 0, Pi1_dict, Pi1_theo_dict, 
                  Pi11_dict, Pi11_theo_dict, 0, 0, 
                  Pi111_shift_dict,Pi111_shift_theo_dict,
                 0, 0, specs_dict)
    return output

    

Sampling function for non-ergodic case. We can specify noise type to either sample exponentially correlated or 1/f noise. 

In [ ]:
def sampling_flicker_short(N_samples, T, D, t_cycle, omega, reps = 100, 
                           n_corr = 20,phi = np.pi/4, noise_type = 'flicker'):
    '''
    Maybe we will adjust correlation matrix, to get rid of tau, and give other meaning to
    the used parameters
    '''
    
    if noise_type == 'flicker':
        theta_arr, F = sampling_theta_flicker(N_samples = N_samples, T = T, D = D,
                                                    t_cycle = t_cycle, omega = omega,
                                              n_corr =  n_corr,reps = reps, noise = 'arctan')
    else:
        theta_arr, F = sampling_theta(N_samples = N_samples, T = T, D = D,t_cycle = t_cycle, 
                                      n_corr =  n_corr,reps = reps)        
    rnd = np.random.rand(reps, N_samples)

    output = (1/2 * (1 + np.cos(phi + theta_arr)) >= rnd) * 1
    return output, theta_arr
